In [1]:
import sys
# sys.path.append("../")
import os
os.chdir("../")

In [2]:
# %load_ext autoreload
# %autoreload 2

In [3]:
import sys
print(sys.path)

['/raid/taoyang/research/research_everyday/projects/DR/splade/splade/scripts', '/raid/taoyang/research/research_everyday/projects/DR/splade/splade/scripts', '/home/taoyang/research/Tao_lib/Tao_lib', '/home/taoyang/research/Tao_lib/BEL', '/home/taoyang/miniconda3/envs/splade/lib/python38.zip', '/home/taoyang/miniconda3/envs/splade/lib/python3.8', '/home/taoyang/miniconda3/envs/splade/lib/python3.8/lib-dynload', '', '/home/taoyang/miniconda3/envs/splade/lib/python3.8/site-packages', '/home/taoyang/miniconda3/envs/splade/lib/python3.8/site-packages/SPLADE-2.1-py3.8.egg', '/home/taoyang/miniconda3/envs/splade/lib/python3.8/site-packages/BatchExpLaunch-0.0.7-py3.8.egg']


In [4]:
import pytrec_eval

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] ="-1"

In [6]:
import argparse
import json
from splade.evaluation.eval import mrr_k, evaluate


def load_and_evaluate(qrel_file_path, run_file_path, metric,*args, **kwargs):
    with open(qrel_file_path) as reader:
        qrel = json.load(reader)
    with open(run_file_path) as reader:
        run = json.load(reader)
    # for trec, qrel_binary.json should be used for recall etc., qrel.json for NDCG.
    # if qrel.json is used for binary metrics, the binary 'splits' are not correct
    if "TREC" in qrel_file_path:
        assert ("binary" not in qrel_file_path) == (metric == "ndcg" or metric == "ndcg_cut")
    if metric == "mrr_10":
        res = mrr_k(run, qrel, k=10)
        print("MRR@10:", res)
        return {"mrr_10": res}
    else:
        res = evaluate(run, qrel, metric=metric,*args, **kwargs)
        print(metric, "==>", res)
        return res
run="experiments/cocondenser_ensemble_distil/out2/TREC_DL_2019/run.json"
qrels="data/msmarco/TREC_DL_2019/qrel.json"
metric="ndcg_cut"
res=load_and_evaluate(qrels,run,metric,agg=False)


ndcg_cut ==> {'156493': {'ndcg_cut_5': 0.8550077055393716, 'ndcg_cut_10': 0.8347727710785315, 'ndcg_cut_15': 0.8690005115907, 'ndcg_cut_20': 0.8892363940828747, 'ndcg_cut_30': 0.8903459863148391, 'ndcg_cut_100': 0.8456686930864262, 'ndcg_cut_200': 0.8616580574038185, 'ndcg_cut_500': 0.9080776185302061, 'ndcg_cut_1000': 0.9353519759599007}, '1110199': {'ndcg_cut_5': 0.8008157892980455, 'ndcg_cut_10': 0.8559744731298657, 'ndcg_cut_15': 0.8633110478805224, 'ndcg_cut_20': 0.7593214811183402, 'ndcg_cut_30': 0.6812161237801484, 'ndcg_cut_100': 0.7513523120199341, 'ndcg_cut_200': 0.8071607930105618, 'ndcg_cut_500': 0.8173835461142229, 'ndcg_cut_1000': 0.8265029855898373}, '1063750': {'ndcg_cut_5': 0.7345577848667817, 'ndcg_cut_10': 0.839758543431562, 'ndcg_cut_15': 0.8702436132800725, 'ndcg_cut_20': 0.8750719642322907, 'ndcg_cut_30': 0.8590294134716118, 'ndcg_cut_100': 0.6047392488251477, 'ndcg_cut_200': 0.46676931377556363, 'ndcg_cut_500': 0.4990821187824705, 'ndcg_cut_1000': 0.5552268914382

In [34]:
resNdcg={}
poorQid=[]
poorQidNew=[]
count=0
for key in res:
    resNdcg[key]=res[key]['ndcg_cut_10']
    if resNdcg[key]<0.6:
        poorQid.append(key)
        print(resNdcg[key])
        poorQidNew.append(count)
        print(key,count)
    count+=1

0.27138809734266756
489204 4
0.5416687420249771
527433 8
0.1528664175953766
1037798 9
0.5898800525573265
148538 15
0.5932499685451033
451602 16
0.577612984431169
1112341 19
0.3959824221672385
1124210 27
0.4917459781954087
490595 28
0.3301376494471202
443396 31
0.4913448133999967
87452 33
0.544828290841365
19335 36
0.547720786297593
47923 37
0.4844914014356644
1113437 40


In [8]:
from splade.datasets.dataloaders import CollectionDataLoader
from splade.datasets.datasets import CollectionDatasetPreLoad
from splade.utils.utils import get_dataset_name, get_initialize_config
from splade.utils.metrics import truncate_run

In [9]:
from splade.tasks.amp import NullContextManager
from splade.utils.utils import generate_bow, normalize

In [10]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer
from splade.models.transformer_rep import Splade, DebugSplade

In [11]:

model_type_or_dir = "experiments/psuedo/checkpoint/model"
# /home/collab/u1368791/largefiles/TaoFiles/splade/experiments/cocondenser_ensemble_distil/checkpoint/model
model = DebugSplade(model_type_or_dir, agg="max")
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_type_or_dir)
reverse_voc = {v: k for k, v in tokenizer.vocab.items()}

In [12]:
from argparse import Namespace

In [13]:
from types import SimpleNamespace

def dict_to_namespace(d):
    if isinstance(d, dict):
        return SimpleNamespace(**{k: dict_to_namespace(v) for k, v in d.items()})
    elif isinstance(d, list):
        return [dict_to_namespace(x) for x in d]
    else:
        return d

In [40]:
def get_initialize_config(exp_dict=None, train=False):
    # delay import to reduce dependencies
    # from ..utils.hydra import hydra_chdir
    # hydra_chdir(exp_dict)
    # exp_dict["init_dict"]["fp16"] = exp_dict["config"].get("fp16", False)
    config = exp_dict["config"]
    init_dict = exp_dict["init_dict"]
    # if train:
    #     os.makedirs(exp_dict.config.checkpoint_dir, exist_ok=True)
    #     OmegaConf.save(config=exp_dict, f=os.path.join(exp_dict.config.checkpoint_dir, "config.yaml"))
    model_training_config = None
    # else:
    #     if config.pretrained_no_yamlconfig:
    #         model_training_config = config
    #     else:
    #         model_training_config = OmegaConf.load(os.path.join(config["checkpoint_dir"], "config.yaml"))["config"]
    return exp_dict, config, init_dict, model_training_config

In [41]:
# # Program to convert yaml file to dictionary
import yaml
# opening a file
yamlFile="/home/taoyang/research/research_everyday/projects/DR/splade/splade/experiments/psuedo/checkpoint/config.yaml"
with open(yamlFile, 'r') as stream:
    # try:
    # Converts yaml document to python object
    exp_dictOrig=yaml.safe_load(stream)
# exp_dictOrig=Namespace(exp_dictOrig)
exp_dict, config, init_dict, model_training_config = get_initialize_config(exp_dictOrig)

In [42]:
collection="data/msmarco/full_collection"
q_collection="data/msmarco/TREC_DL_2019/queries_2019"
collection = CollectionDatasetPreLoad(data_dir=collection, id_style="content_id")

q_collection = CollectionDatasetPreLoad(data_dir=q_collection, id_style="content_id")
# loader = CollectionDataLoader(dataset=q_collection, tokenizer_type="Luyu/co-condenser-marco",
#                                 max_length=128, batch_size=1,
#                                 shuffle=False, num_workers=1)

KeyboardInterrupt: 

In [17]:
from splade.tasks.transformer_evaluator import SparseRetrieval,DeubgRetrieval
import json
import os
import pickle
import time
from collections import defaultdict

import numba
import numpy as np
import torch
from tqdm.auto import tqdm
import psutil
from splade.indexing.inverted_index import IndexDictOfArray
from splade.losses.regularization import L0
from splade.tasks.base.evaluator import Evaluator
from splade.utils.utils import makedir, to_list


In [18]:
model2=DebugSplade(**init_dict)


In [19]:
config["checkpoint_dir"]

'experiments/cocondenser_ensemble_distil/checkpoint'

In [23]:
from splade.utils.utils import restore_model

In [43]:
model_type_or_dir = "experiments/psuedo/checkpoint/model"
# /home/collab/u1368791/largefiles/TaoFiles/splade/experiments/cocondenser_ensemble_distil/checkpoint/model
model = DebugSplade(model_type_or_dir, agg="max")
model.eval()
checkpoint = torch.load(os.path.join(config["checkpoint_dir"], "model/model.tar"),
                                            map_location="cpu")
restore_model(model, checkpoint["model_state_dict"])

restoring model: DebugSplade


In [24]:
data_dir="data/msmarco/TREC_DL_2019/queries_2019"
# os.chdir("splade")
evaluator = DeubgRetrieval(config=config, model=model, dataset_name=get_dataset_name(data_dir),
                                    compute_stats=True, dim_voc=model.output_dim)
# os.chdir("../")


# evaluator.model=model

avalialbe devices is cpu, count of devices is 0
restoring model: DebugSplade
restore model on CPU at experiments/cocondenser_ensemble_distil/checkpoint/model/model.tar
index already exists, loading...


  0%|          | 0/30522 [00:00<?, ?it/s]

done loading index...


In [26]:
evaluator.model=model

In [ ]:
runFile="experiments/cocondenser_ensemble_distil/out2/TREC_DL_2019/run.json"
with open(runFile) as reader:
        run = json.load(reader)
runTop10=truncate_run(run,k=10)
qrel_file_path="data/msmarco/TREC_DL_2019/qrel.json"
with open(qrel_file_path) as reader:
        qrel = json.load(reader)
qrelTop10=truncate_run(qrel,k=10)

In [28]:
poorQid

In [ ]:
def converziptoDict(zipVar):
    res={}
    for i,j,k in zipVar:
        res[i]=(j,k)
    return res

In [44]:
poorQid=sorted(resNdcg,key=lambda x:resNdcg[x])

In [ ]:
from splade.utils.utils import makedir, to_list
import numpy as np
from collections import defaultdict
import re

In [ ]:
q_id='1037798'
queryText=q_collection.data_dict[q_id]

In [ ]:
poorQid[16:25]

['131843',
 '182539',
 '915593',
 '1129237',
 '405717',
 '1133167',
 '1103812',
 '1121709',
 '359349']

In [ ]:
# from nltk.corpus import wordnet
# import nltk

In [ ]:
# import spacy
# from spacy import displacy
# from spacy import tokenizer
# nlp = spacy.load('en_core_sci_sm')
# text =("why did the united states volunterilay enter ww1")
# # text2 = # copy the paragraphs from  https://www.python.org/doc/essays/
# doc = nlp(text)
# #doc2 = nlp(text2)
# sentences = list(doc.sents)
# print(sentences)
# # tokenization
# for token in doc:
#     print(token.text)
# # print entities
# ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
# print(ents)
# # now we use displaycy function on doc2
# displacy.render(doc, style='ent', jupyter=True)

[why did the united states volunterilay enter ww1]
why
did
the
united
states
volunterilay
enter
ww1
[('united states', 12, 25, 'ENTITY'), ('volunterilay enter', 26, 44, 'ENTITY'), ('ww1', 45, 48, 'ENTITY')]


In [ ]:
# doc = nlp(text)
# #doc2 = nlp(text2)
# sentences = list(doc.sents)
# print(sentences)
# # tokenization
# for token in doc:
#     print(token.text)
# # print entities
# ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
# displacy.render(doc, style='ent', jupyter=True)

[why did the united states volunterilay enter ww1]
why
did
the
united
states
volunterilay
enter
ww1


In [ ]:
# from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
# print(len(stop_words), "stopwords:", stop_words)

In [ ]:
# import spacy
# nlp = spacy.load("en_core_web_sm")

In [ ]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

# from nltk import pos_tag, word_tokenize, RegexpParser
# text = "Reliance Retail acquires majority stake in designer brand Abraham & Thakore."
# tokens = word_tokenize(text)
# tags = pos_tag(tokens)
# print(tags)

In [ ]:
# for qid in poorQid:
#     print("*"*100)
#     query=q_collection.data_dict[qid]
#     print(qid,query)
#     doc = nlp(query)
#     ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
#     print(ents)
#     tokens = word_tokenize(query)
#     tags = pos_tag(tokens)
#     print(tags)


In [ ]:
idfpkl="output/idf-token.pkl"
with open(idfpkl, 'rb') as f:
    idf = pickle.load(f)

In [33]:
poorQid

['489204',
 '527433',
 '1037798',
 '148538',
 '451602',
 '1112341',
 '1124210',
 '490595',
 '443396',
 '87452',
 '19335',
 '47923',
 '1113437']

In [47]:
q_id= '443396'
# "the query is :who is robert gray"
queryText=q_collection.data_dict[q_id]
# queryText="who"
queryText="who is robert gray"
print(f"the query is :{queryText}")
top_k=10

queryExplicit,query=outputEmb(queryText,model=model,returnRep=True)
print(queryExplicit)

the query is :who is robert gray
[('is', 2.51), ('.', 2.46), ('the', 2.42), ('robert', 2.36), ('and', 2.35), ('it', 2.33), ('in', 2.31), ('that', 2.27), ('of', 2.22), ('?', 2.17), ('to', 2.17), ('(', 2.15), (',', 2.09), ('for', 2.04), ('by', 2.0), ('a', 1.99), ('this', 1.97), ('you', 1.91), (':', 1.89), ('he', 1.89), ('as', 1.87), ('or', 1.86), ('was', 1.85), ('at', 1.84), ('from', 1.84), ('!', 1.79), ('on', 1.79), ('-', 1.74), ('"', 1.74), ('with', 1.74), ('who', 1.73), (')', 1.73), ('are', 1.71), ('what', 1.71), ('but', 1.69), ('which', 1.64), (';', 1.63), ('there', 1.63), ('gray', 1.61), ('an', 1.51), ('its', 1.48), ('if', 1.48), ('they', 1.4), ('when', 1.4), ('your', 1.38), ('be', 1.36), ("'", 1.34), ('also', 1.29), ('she', 1.19), ('where', 1.1), ('bob', 1.08), ('then', 1.05), ('one', 1.02), ('without', 1.0), ('grey', 0.99), ('i', 0.98), ('so', 0.96), ('morton', 0.9), ('how', 0.87), ('bobby', 0.77), ('into', 0.76), ('his', 0.73), ('richard', 0.69), ('bruce', 0.62), ('thomas', 0.61)

/home/taoyang/miniconda3/envs/splade/lib/python3.8/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
q_id= '443396'
# "the query is :who is robert gray"
queryText=q_collection.data_dict[q_id]
# queryText="who"

print(f"the query is :{queryText}")
top_k=10

queryExplicit,query=outputEmb(queryText,model=model,returnRep=True)
print(queryExplicit)
query=query[None, :]
runCur = defaultdict(dict)
row, col = torch.nonzero(query, as_tuple=True)
values = query[to_list(row), to_list(col)]
# argmax= output["argmax"]
# group=argmax[to_list(row), to_list(col)].cpu().numpy()
filtered_indexes, scores = evaluator.numba_score_float(evaluator.numba_index_doc_ids,
                                                    evaluator.numba_index_doc_values,
                                                    col.cpu().numpy(),
                                                    values.cpu().numpy().astype(np.float32),
                                                    threshold=0,
                                                    size_collection=evaluator.sparse_index.nb_docs())
# threshold set to 0 by default, could be better
filtered_indexes, scores = evaluator.select_topk(filtered_indexes, scores, k=top_k)
argsortedId=np.argsort(-scores)
filtered_indexes, scores=filtered_indexes[argsortedId], scores[argsortedId]
for id_, sc in zip(filtered_indexes, scores):
    runCur[str(q_id)][str(evaluator.doc_ids[id_])] = float(sc)
CurMetr = evaluate(runCur, qrel, metric=metric)['ndcg_cut_10']
print(f"the ndcg@10 is {CurMetr}")


# for the text to have robert gray
# runCur = defaultdict(dict)
# for id_, sc in zip(filtered_indexes, scores):
#     doc=collection.data_dict[str(id_)]
#     doc=doc.lower()
#     # if "robert gray" not in doc:
#     # flag=r"robert gray"
#     # # result =re.match(flag,doc) is None
#     # result =flag not in doc
    
#     # flag=r"robert gray[ ,.?]{1}"
#     # flag=r"robert.*gray"
#     # result =len(re.findall(flag,doc))==0
#     # result="medic" not in doc or "mechani" not in doc  or "venti" not in doc
#     result="us" not in doc 
#     if result:
#         sc=-100
#     # else:
#     #     print(doc)
#     # print(doc)
#     # print(id_)
#     runCur[str(q_id)][str(evaluator.doc_ids[id_])] = float(sc)
# CurMetr = evaluate(runCur, qrel, metric=metric)['ndcg_cut_10']
# print(f"the ndcg@10 is {CurMetr}")


the query is :lps laws definition
[('lp', 3.17), ('##s', 1.82), ('law', 1.55), ('laws', 1.41), ('l', 1.34), ('definition', 1.24), ('ian', 0.74), ('legal', 0.73), ('officer', 0.7), ('regulation', 0.67), ('lane', 0.56), ('garrett', 0.55), ('davis', 0.5), ('ambulance', 0.43), ('trent', 0.43), ('dean', 0.4), ('corey', 0.35), ('drug', 0.35), ('wes', 0.35), ('lc', 0.33), ('dick', 0.32), ('eli', 0.28), ('abbreviation', 0.28), ('policy', 0.26), ('craig', 0.26), ('statute', 0.25), ('act', 0.24), ('council', 0.24), ('reed', 0.24), ('hart', 0.22), ('eva', 0.22), ('police', 0.21), ('morris', 0.21), ('technology', 0.19), ('label', 0.17), ('miller', 0.17), ('nurse', 0.16), ('dana', 0.13), ('wagner', 0.13), ('registration', 0.12), ('levi', 0.12), ('hp', 0.11), ('is', 0.11), ('sip', 0.08), ('cooper', 0.08), ('partnership', 0.08), ('organization', 0.07), ('reid', 0.05), ('frank', 0.05), ('avery', 0.04), ('phil', 0.04), ('gary', 0.04), ('license', 0.04), ('lung', 0.02), ('beck', 0.02), ('cam', 0.01), ('

In [ ]:
tokenizerDict={reverse_voc[key]:key for key in reverse_voc}

In [ ]:
docs=[collection.data_dict[str(d_i)]for d_i in filtered_indexes]
tokens=tokenizer(docs, return_tensors="pt",padding=True)['input_ids'].cpu().numpy()

In [ ]:
docfreqq=np.zeros(tokenizer.vocab_size)
copfreqq=np.zeros(tokenizer.vocab_size)

In [ ]:
np.add.at(docfreqq,tokens,1)

In [ ]:
for token in idf:
    copfreqq[tokenizerDict[token]] =idf[token] 

In [ ]:
diff=docfreqq-copfreqq/len(collection.data_dict)*100

In [ ]:
copfreqq[tokenizerDict[ 'the']]/len(collection.data_dict)*100

87.31969640197502

In [ ]:
docfreqq[tokenizerDict[ 'the']]

20.0

In [ ]:
argSort=np.argsort(-diff)[:100]
importantWords=[reverse_voc[i] for i in argSort]
print(importantWords)

['[PAD]', 'lp', 'limited', 'partnership', 'act', 'lip', 'partner', 'general', 'legal', 'attorney', 'outer', 'core', 'outs', '##ris', '##our', 'lan', 'law', 'pet', '##man', 'least', '##ter', '–', '##cans', '##ride', '##xin', '##valent', '##hari', 'antigen', 'eli', '##oto', 'responses', '##cit', 'commitment', '##cing', 'liability', 'gram', 'unable', 'consisting', 'joined', 'persons', '##cha', 'bond', '##cc', 'membrane', 'inner', 'mental', 'composed', 'immune', '##ys', 'molecules', 'short', 'negative', 'protection', 'poly', '##ol', '##og', '##ac', '##sa', 'strong', 'animals', 'bacteria', '##de', '##id', 'california', '##op', 'gravely', 'empowered', 'precedent', 'judged', 'sponsors', 'gps', 'organisation', 'costly', 'involuntary', 'damaging', 'conducting', 'lightning', '5000', '##lf', 'shelter', 'creatures', 'areas', 'statute', 'lighting', 'behalf', 'unlimited', 'appointed', 'disabled', '##cted', 'sec', '##mba', 'provides', 'legislation', 'legally', 'departments', 'lasting', 'decreased', '

In [ ]:
diff[argSort]

array([185.99989821,  14.90861613,   8.40314345,   7.8877607 ,
         5.84111003,   5.72965982,   4.78495385,   4.30959068,
         4.2751721 ,   3.80195261,   3.69536825,   3.62935245,
         2.94315652,   2.9162503 ,   2.894049  ,   2.88619994,
         2.71632807,   2.64864712,   2.47114978,   2.41774711,
         2.34208138,   2.        ,   1.97797965,   1.94924124,
         1.93633666,   1.92830664,   1.92465355,   1.91760749,
         1.911896  ,   1.90720239,   1.89068996,   1.89061079,
         1.89061079,   1.84054193,   1.83403875,   1.82260446,
         1.81993532,   1.80896473,   1.80605809,   1.76855452,
         1.74248523,   1.74117329,   1.72177683,   1.70055949,
         1.68126483,   1.66054512,   1.65479969,   1.64916737,
         1.63750688,   1.63676043,   1.61367955,   1.58119496,
         1.54742365,   1.54173478,   1.52657953,   1.46050718,
         1.45421889,   1.43791003,   1.35378711,   1.3297536 ,
         1.32249266,   1.26701767,   1.25283508,   1.11

In [ ]:
freqterm={key:0 for key in idf}


for doc in filtered_indexes:
    tokenList=tokenizer(doc, return_tensors="pt")["input_"]
    for token in tokenList:
        freqterm[token]+=1

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
# from rake_nltk import Rake

# # Uses stopwords for english from NLTK, and all puntuation characters by
# # default
# r = Rake()
# sent=""
# # Extraction given the text.
# r.extract_keywords_from_text()

# # Extraction given the list of strings where each string is a sentence.
# r.extract_keywords_from_sentences(<list of sentences>)

# # To get keyword phrases ranked highest to lowest.
# r.get_ranked_phrases()

# # To get keyword phrases ranked highest to lowest with scores.
# r.get_ranked_phrases_with_scores()

In [ ]:
for qid in runCur:
    print("*"*100)
    query=q_collection.data_dict[qid]
    query=query
    q_emb=outputEmb_source(query,model=model)
    q_embDict=converziptoDict(q_emb)
    print("query is --",query,f"ndcg is {resNdcg[qid]}",q_emb)
    # print(outputEmb("who are you"))
    for d_i in runCur[qid]:
        doc=collection.data_dict[d_i]
        doc_emb=outputEmb_source(doc,model=model)
        d_embDict=converziptoDict(doc_emb)
        print(doc,doc_emb)
        inter=list(set(q_embDict.keys())&set(d_embDict.keys()))
        interTrip=[(intersec,q_embDict[intersec],d_embDict[intersec])for intersec in inter]
        interTrip=sorted(interTrip,key=lambda x: -x[1][1])
        print(f"the intersection is {interTrip}")
        rel=qrel[qid][d_i] if d_i in qrel[qid] else 0
        print(f"its score is {runCur[qid][d_i]}, its relevance is {rel}, the doc id in {d_i}","*"*10)
    # print("##"*10)
    # for d_i in qrelTop10[qid]:
    #     doc=collection.data_dict[d_i]
    #     doc_emb=outputEmb_source(doc,model=model)
    #     d_embDict=converziptoDict(doc_emb)
    #     print(doc,doc_emb)
    #     inter=list(set(q_embDict.keys())&set(d_embDict.keys()))
    #     interTrip=[(intersec,q_embDict[intersec],d_embDict[intersec])for intersec in inter]
    #     interTrip=sorted(interTrip,key=lambda x: -x[1][1])
    #     print(f"the intersection is {interTrip}")        
    #     rel=qrel[qid][d_i] if d_i in qrel[qid] else 0
    #     score=runCur[qid][d_i] if d_i in runCur[qid] else "not in top {top_k}"
    #     print(f"its score is {score}, its relevance is {rel}, the doc id in {d_i}","*"*10)
    

****************************************************************************************************
query is -- why did the us volunterilay enter ww1 ndcg is 0.839758543431562 [('because', 'why', 1.84), ('war', '##w', 1.47), ('us', 'us', 1.43), ('enter', 'enter', 1.36), ('##w', '##w', 1.34), ('wwii', '##w', 1.27), ('vol', 'vol', 1.27), ('w', 'w', 1.25), ('##unt', '##unt', 1.01), ('army', 'the', 0.95), ('united', 'us', 0.9), ('was', 'the', 0.9), ('helped', 'did', 0.86), ('why', 'why', 0.79), ('##1', '##1', 0.76), ('join', 'enter', 0.76), ('entry', 'enter', 0.73), ('conflict', 'did', 0.62), ('germany', 'the', 0.55), ('neutrality', 'did', 0.54), ('##lay', '##lay', 0.51), ('goal', 'did', 0.46), ('slavery', '[CLS]', 0.44), ('##eri', '##eri', 0.42), ('became', '##lay', 0.41), ('england', '##1', 0.4), ('battle', '[CLS]', 0.39), ('were', '##lay', 0.37), ('started', 'did', 0.36), ('i', '##1', 0.35), ('caused', 'why', 0.33), ('1', '##1', 0.33), ('american', 'us', 0.28), ('confederate', 'the', 0

KeyboardInterrupt: 

In [ ]:
CurMetr = evaluate(runCur, qrel, metric=metric)['ndcg_cut_10']
print(f"the ndcg@10 is {CurMetr}")

the ndcg@10 is 0.8484359194787117


In [ ]:
queryExplicit

[('because', 1.84),
 ('war', 1.47),
 ('us', 1.43),
 ('enter', 1.36),
 ('##w', 1.34),
 ('wwii', 1.27),
 ('vol', 1.27),
 ('w', 1.25),
 ('##unt', 1.01),
 ('army', 0.95),
 ('united', 0.9),
 ('was', 0.9),
 ('helped', 0.86),
 ('why', 0.79),
 ('##1', 0.76),
 ('join', 0.76),
 ('entry', 0.73),
 ('conflict', 0.62),
 ('germany', 0.55),
 ('neutrality', 0.54),
 ('##lay', 0.51),
 ('goal', 0.46),
 ('slavery', 0.44),
 ('##eri', 0.42),
 ('became', 0.41),
 ('england', 0.4),
 ('battle', 0.39),
 ('were', 0.37),
 ('started', 0.36),
 ('i', 0.35),
 ('caused', 0.33),
 ('1', 0.33),
 ('american', 0.28),
 ('confederate', 0.26),
 ('military', 0.25),
 ('##ord', 0.2),
 ('result', 0.15),
 ('had', 0.14),
 ('event', 0.13),
 ('imperialism', 0.12),
 ('disbanded', 0.1),
 ('treaty', 0.09),
 ('##and', 0.07),
 ('downfall', 0.07),
 ('decision', 0.07),
 ('invasion', 0.07),
 ('garrison', 0.06),
 ('country', 0.06),
 ('success', 0.06),
 ('independence', 0.05),
 ('didn', 0.04),
 ('1945', 0.02),
 ('advantage', 0.02),
 ('europe', 0

In [ ]:
queryText="robert gray"

print(f"the query is :{queryText}")
top_k=100000
queryExplicit,query=outputEmb(queryText,model=model,returnRep=True)
query=query[None, :]
runCur = defaultdict(dict)
row, col = torch.nonzero(query, as_tuple=True)
values = query[to_list(row), to_list(col)]
# argmax= output["argmax"]
# group=argmax[to_list(row), to_list(col)].cpu().numpy()
filtered_indexes, scores = evaluator.numba_score_float(evaluator.numba_index_doc_ids,
                                                    evaluator.numba_index_doc_values,
                                                    col.cpu().numpy(),
                                                    values.cpu().numpy().astype(np.float32),
                                                    threshold=0,
                                                    size_collection=evaluator.sparse_index.nb_docs())
# threshold set to 0 by default, could be better
filtered_indexes, scores = evaluator.select_topk(filtered_indexes, scores, k=top_k)
argsortedId=np.argsort(-scores)
filtered_indexes, scores=filtered_indexes[argsortedId], scores[argsortedId]
for id_, sc in zip(filtered_indexes, scores):
    runCur[str(q_id)][str(evaluator.doc_ids[id_])] = float(sc)

the query is :robert gray


In [ ]:
doc="captain robert, may 1972. gray discovering the columbia river, may 17"
doc=doc.lower()
flag=r"robert.*gray"
result =len(re.findall(flag,doc))==0
print(result)

False


In [37]:
# outputEmb("gray")
queryText="gray robert"
queryExplicit=outputEmb(queryText,model=model)
print(queryExplicit)

[('robert', 2.99), ('gray', 2.94), ('grey', 1.99), ('he', 0.75), ('actor', 0.63), ('color', 0.6), ('wagner', 0.55), ('who', 0.53), ('bobby', 0.52), ('brook', 0.43), ('his', 0.37), ('personality', 0.26), ('kim', 0.26), ('booth', 0.25), ('walter', 0.24), ('brien', 0.23), ('jerry', 0.2), ('anderson', 0.2), ('character', 0.19), ('davis', 0.17), ('terry', 0.16), ('white', 0.16), ('hawkins', 0.14), ('baxter', 0.13), ('lang', 0.12), ('bruce', 0.11), ('family', 0.09), ('breed', 0.08), ('here', 0.07), ('james', 0.07), ('black', 0.07), ('gay', 0.06), ('brown', 0.06), ('wearing', 0.05), ('park', 0.05), ('chip', 0.04), ('avery', 0.04), ('wright', 0.03), ('nixon', 0.02), ('.', 0.02), ('robinson', 0.01), ('charlie', 0.0)]


In [ ]:
for id_, sc in zip(filtered_indexes, scores):
    doc=collection.data_dict[str(id_)]
    doc=doc.lower()
    # flag=r"actor"
    # result =len(re.findall(flag,doc))>0
    result="gray" in doc
    if result:
        print(doc)
    

In [ ]:
for docid in filtered_indexes:
    doc=collection.data_dict[str(docid)]
    print(doc)

In [ ]:
qids=['1113437']
for qid in poorQid[0:3]:
# for qid in qids:
    print("*"*100)
    query=q_collection.data_dict[qid]
    query=query
    q_emb=outputEmb_source(query,model=model)
    q_embDict=converziptoDict(q_emb)
    print("query is --",query,f"ndcg is {resNdcg[qid]}",q_emb)
    # print(outputEmb("who are you"))
    for d_i in runTop10[qid]:
        doc=collection.data_dict[d_i]
        doc_emb=outputEmb_source(doc,model=model)
        d_embDict=converziptoDict(doc_emb)
        print(doc,doc_emb)
        inter=list(set(q_embDict.keys())&set(d_embDict.keys()))
        interTrip=[(intersec,q_embDict[intersec],d_embDict[intersec])for intersec in inter]
        interTrip=sorted(interTrip,key=lambda x: -x[1][1])
        print(f"the intersection is {interTrip}")
        rel=qrel[qid][d_i] if d_i in qrel[qid] else 0
        print(f"its score is {runTop10[qid][d_i]}, its relevance is {rel}, the doc id in {d_i}","*"*10)
    print("##"*10)
    for d_i in qrelTop10[qid]:
        doc=collection.data_dict[d_i]
        doc_emb=outputEmb_source(doc,model=model)
        d_embDict=converziptoDict(doc_emb)
        print(doc,doc_emb)
        inter=list(set(q_embDict.keys())&set(d_embDict.keys()))
        interTrip=[(intersec,q_embDict[intersec],d_embDict[intersec])for intersec in inter]
        interTrip=sorted(interTrip,key=lambda x: -x[1][1])
        print(f"the intersection is {interTrip}")        
        rel=qrel[qid][d_i] if d_i in qrel[qid] else 0
        score=run[qid][d_i] if d_i in run[qid] else "not in top 1000"
        print(f"its score is {score}, its relevance is {rel}, the doc id in {d_i}","*"*10)

****************************************************************************************************
query is -- who is robert gray ndcg is 0.1528664175953766 [('gray', 'gray', 3.12), ('robert', 'robert', 3.04), ('grey', 'gray', 2.2), ('who', 'who', 1.79), ('actor', 'who', 1.52), ('barr', 'gray', 1.07), ('he', 'is', 1.06), ('his', 'is', 1.01), ('kent', 'gray', 0.6), ('whom', 'who', 0.57), ('terry', '[CLS]', 0.29), ('him', '[CLS]', 0.28), ('whose', 'who', 0.27), ('wagner', '[CLS]', 0.21), ('nixon', '[CLS]', 0.12), ('davis', '[CLS]', 0.11), ('writer', '[CLS]', 0.1), ('bobby', '[CLS]', 0.08), ('personality', '[CLS]', 0.07), ('kim', '[CLS]', 0.05), ('architect', 'who', 0.05), ('brien', '[CLS]', 0.01), ('bruce', '[CLS]', 0.01)]
Robert Grey (born 21 April 1951 in Marefield, Leicestershire) is an English musician best known as the drummer for Wire. In 1973, Grey joined his first band, an R&B group called the Snakes, as vocalist. The Snakes released one single: Teenage Head.. After the group f